# Análise e Reorganização da Estrutura do Projeto Decision Scoring

Este notebook analisa a estrutura atual do projeto Decision Scoring e propõe melhorias de organização seguindo boas práticas de desenvolvimento Python.

## Objetivos

1. Analisar a estrutura atual do projeto
2. Identificar problemas organizacionais
3. Propor uma estrutura padrão de diretórios
4. Criar um plano detalhado de reorganização
5. Implementar estratégias para atualização de referências e documentação

In [ ]:
# Importando bibliotecas necessárias
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from collections import Counter, defaultdict
import re
import shutil
from IPython.display import display, Markdown, HTML

## 1. Análise da Estrutura Atual do Projeto

Primeiro, vamos analisar a estrutura atual do projeto para entender como ele está organizado.

In [ ]:
# Definindo o diretório raiz do projeto
project_root = '/Users/calicojack/Development/4MLET/fiap_tech_challenge_05'
os.chdir(project_root)
print(f"Diretório raiz do projeto: {project_root}")

# Função para escanear a estrutura do projeto
def scan_project_structure(root_dir, max_depth=None, exclude_dirs=None):
    if exclude_dirs is None:
        exclude_dirs = ['.git', '.venv', '__pycache__', '.ipynb_checkpoints']
    
    file_list = []
    
    for root, dirs, files in os.walk(root_dir):
        # Excluindo diretórios
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        # Verificando a profundidade
        rel_path = os.path.relpath(root, root_dir)
        depth = 0 if rel_path == '.' else len(rel_path.split(os.sep))
        
        if max_depth is not None and depth > max_depth:
            continue
        
        for file in files:
            file_path = os.path.join(root, file)
            rel_file_path = os.path.relpath(file_path, root_dir)
            
            file_info = {
                'path': file_path,
                'rel_path': rel_file_path,
                'name': file,
                'ext': os.path.splitext(file)[1],
                'size': os.path.getsize(file_path),
                'depth': depth,
                'is_root': (depth == 0)
            }
            
            file_list.append(file_info)
    
    return pd.DataFrame(file_list)

# Escaneando a estrutura do projeto
df_files = scan_project_structure(project_root)

# Exibindo um resumo
print(f"Total de arquivos encontrados: {len(df_files)}")
print("\nDistribuição de arquivos por tipo:")
ext_counts = df_files['ext'].value_counts()
display(ext_counts.head(10))

# Arquivos no diretório raiz
root_files = df_files[df_files['is_root']].sort_values('name')
print(f"\nArquivos no diretório raiz ({len(root_files)}):")
display(root_files[['name', 'ext', 'size']])

In [ ]:
# Visualizando a distribuição de arquivos por diretório
dir_counts = df_files['rel_path'].apply(lambda x: os.path.dirname(x) or 'raiz').value_counts()

plt.figure(figsize=(12, 8))
sns.barplot(x=dir_counts.values, y=dir_counts.index)
plt.title('Distribuição de Arquivos por Diretório')
plt.xlabel('Número de Arquivos')
plt.tight_layout()
plt.show()

# Analisando a distribuição de tipos de arquivo por diretório
dir_ext_data = df_files.groupby(df_files['rel_path'].apply(lambda x: os.path.dirname(x) or 'raiz'))['ext'].value_counts().unstack().fillna(0)

# Selecionando os diretórios com mais arquivos
top_dirs = dir_counts.head(10).index
top_dir_ext_data = dir_ext_data.loc[dir_ext_data.index.isin(top_dirs)]

plt.figure(figsize=(14, 10))
top_dir_ext_data.plot(kind='bar', stacked=True, figsize=(14, 8))
plt.title('Tipos de Arquivo por Diretório')
plt.xlabel('Diretório')
plt.ylabel('Número de Arquivos')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Extensão', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 2. Identificando Problemas de Organização

Vamos analisar os arquivos na raiz e identificar possíveis problemas organizacionais.

In [ ]:
# Classificando arquivos da raiz por categoria
def classify_root_file(file_name, file_ext):
    # Configurações
    if file_ext in ['.yml', '.yaml', '.env', '.ini', '.cfg'] or file_name in ['Procfile', 'Dockerfile', 'docker-compose.yml']:
        return 'Configuração'
    
    # Código Python
    elif file_ext == '.py':
        return 'Código Python'
    
    # Scripts
    elif file_ext == '.sh':
        return 'Script'
    
    # Documentação
    elif file_ext in ['.md', '.txt', '.rst']:
        return 'Documentação'
    
    # Temporários
    elif 'test_report' in file_name or file_ext in ['.log', '.tmp']:
        return 'Arquivo temporário'
    
    # Git
    elif file_name.startswith('.git') or file_name in ['.gitignore', '.gitattributes']:
        return 'Controle de versão'
    
    # Outros
    return 'Outros'

# Aplicando a classificação
root_files['categoria'] = root_files.apply(lambda row: classify_root_file(row['name'], row['ext']), axis=1)

# Contando por categoria
category_counts = root_files['categoria'].value_counts()

# Visualizando distribuição de categorias
plt.figure(figsize=(10, 6))
sns.barplot(x=category_counts.values, y=category_counts.index)
plt.title('Tipos de Arquivos na Raiz do Projeto')
plt.xlabel('Número de Arquivos')
plt.tight_layout()
plt.show()

# Listando arquivos por categoria
for categoria in category_counts.index:
    print(f"\n### {categoria} ({len(root_files[root_files['categoria'] == categoria])}):")
    display(root_files[root_files['categoria'] == categoria][['name', 'size']])

### Problemas Identificados

Baseado na análise dos arquivos na raiz do projeto, podemos identificar os seguintes problemas de organização:

1. **Arquivos de configuração dispersos**: Múltiplos arquivos de configuração diretamente na raiz (Docker, Nginx, configurações de ambiente)
2. **Scripts utilitários na raiz**: Scripts shell que deveriam estar no diretório `scripts` 
3. **Código Python na raiz**: Arquivos `.py` que deveriam estar em módulos específicos
4. **Arquivos temporários e logs**: Arquivos como logs e relatórios de teste que deveriam estar em diretórios específicos
5. **Falta de padrão**: Não há uma estrutura clara para onde cada tipo de arquivo deveria estar

Vamos criar um plano para reorganizar esses arquivos seguindo boas práticas de projeto Python.

## 3. Propondo uma Estrutura Padrão de Diretórios

Vamos definir uma estrutura de diretórios padrão seguindo as melhores práticas para projetos Python.

In [ ]:
# Definindo a estrutura padrão de diretórios
standard_structure = {
    '.github': {
        'description': 'GitHub Actions e templates para issues/PRs',
        'subdirs': {
            'workflows': 'Configurações CI/CD'
        }
    },
    'config': {
        'description': 'Arquivos de configuração',
        'subdirs': {
            'docker': {
                'description': 'Arquivos relacionados ao Docker',
                'subdirs': {
                    'api': 'Configuração Docker para API',
                    'dashboard': 'Configuração Docker para Dashboard'
                }
            },
            'nginx': 'Configuração do Nginx',
            'render': 'Configuração específica do Render'
        }
    },
    'data': {
        'description': 'Dados do projeto (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'docs': {
        'description': 'Documentação (já organizado)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'examples': {
        'description': 'Exemplos (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'logs': {
        'description': 'Logs da aplicação',
        'subdirs': {}
    },
    'models': {
        'description': 'Modelos treinados (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'notebooks': {
        'description': 'Jupyter notebooks (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'scripts': {
        'description': 'Scripts utilitários (já reorganizado)',
        'subdirs': {
            'utils': 'Scripts utilitários gerais',
            'deployment': 'Scripts de implantação',
            'monitoring': 'Scripts de monitoramento'
        }
    },
    'src': {
        'description': 'Código-fonte principal (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    },
    'tests': {
        'description': 'Testes (mantido como está)',
        'subdirs': {}  # Já existe, manter a estrutura atual
    }
}

# Função para gerar representação visual da estrutura
def print_directory_tree(structure, prefix=''):
    lines = []
    
    for i, (dir_name, content) in enumerate(structure.items()):
        is_last = i == len(structure) - 1
        
        # Prefixo atual
        current_prefix = prefix + ('└── ' if is_last else '├── ')
        
        # Descrição
        desc = f"# {content['description']}" if 'description' in content else ''
        
        # Adicionar linha
        lines.append(f"{current_prefix}{dir_name} {desc}")
        
        # Prefixo para subdiretorios
        sub_prefix = prefix + ('    ' if is_last else '│   ')
        
        # Processar subdiretorios
        if 'subdirs' in content and content['subdirs']:
            if isinstance(content['subdirs'], dict):
                sub_structure = {}
                for sub_dir, sub_desc in content['subdirs'].items():
                    if isinstance(sub_desc, dict):
                        sub_structure[sub_dir] = sub_desc
                    else:
                        sub_structure[sub_dir] = {'description': sub_desc}
                
                lines.extend(print_directory_tree(sub_structure, sub_prefix))
    
    return lines

# Visualizando a estrutura proposta
dir_tree = '\n'.join(print_directory_tree(standard_structure))

print("Estrutura de Diretórios Proposta:")
print("fiap_tech_challenge_05/")
print(dir_tree)

## 4. Plano de Movimentação de Arquivos

Vamos criar um plano detalhado para mover os arquivos da raiz para seus locais apropriados na nova estrutura.

In [ ]:
# Definindo o mapeamento de arquivos da raiz para nova estrutura
file_movement_plan = {
    # Arquivos Docker
    'Dockerfile': 'config/docker/api/Dockerfile',
    'Dockerfile.dashboard': 'config/docker/dashboard/Dockerfile',
    'docker-compose.yml': 'config/docker/docker-compose.yml',
    
    # Configuração Nginx
    'nginx.conf.example': 'config/nginx/nginx.conf',
    
    # Configuração Render
    'render.yaml': 'config/render/render.yaml',
    'Procfile': 'config/render/Procfile',
    '.env.render': 'config/render/.env.render',
    
    # Scripts
    'debug_api.py': 'scripts/utils/debug_api.py',
    'quick_deploy.sh': 'scripts/deployment/quick_deploy.sh',
    'commit-changes.sh': 'scripts/utils/commit-changes.sh',
    'generate_prospects_curl.py': 'scripts/utils/generate_prospects_curl.py',
    
    # Logs
    'api_logs.log': 'logs/api_logs.log',
    
    # Arquivos temporários para remover
    'test_report.html': None,  # Remover
    '*.pyc': None,  # Remover todos .pyc
}

# Criar DataFrame com o plano de movimentação
movement_data = []
for src_file, dest_path in file_movement_plan.items():
    action = 'Mover' if dest_path else 'Remover'
    movement_data.append({
        'Arquivo Original': src_file,
        'Novo Local': dest_path if dest_path else 'N/A',
        'Ação': action
    })

movement_df = pd.DataFrame(movement_data)
display(movement_df)

## 5. Gerenciamento de Configurações

Vamos analisar como centralizar e padronizar os arquivos de configuração.

In [ ]:
# Análise dos arquivos de configuração
config_files = [f for f in root_files['name'] if f.endswith(('.env', '.yml', '.yaml', '.conf', '.cfg', '.ini'))]
config_files += [f for f in root_files['name'] if f in ['Dockerfile', 'Dockerfile.dashboard', 'docker-compose.yml', 'Procfile']]
config_files = sorted(config_files)

print("Arquivos de configuração identificados na raiz:")
for file in config_files:
    print(f"- {file}")

# Proposta de organização de configurações
config_structure = {
    'config': {
        'docker': {
            'api': ['Dockerfile'],
            'dashboard': ['Dockerfile.dashboard'],
            'root': ['docker-compose.yml']
        },
        'nginx': ['nginx.conf.example'],
        'render': ['render.yaml', 'Procfile', '.env.render'],
        'env': ['.env', '.env.example']
    }
}

def print_config_structure(structure, prefix=''):
    result = []
    
    for dir_name, content in structure.items():
        # Adicionar diretório
        result.append(f"{prefix}{dir_name}/")
        
        # Iterar conteúdo
        for key, value in content.items():
            if isinstance(value, dict):
                # É um subdiretório
                sub_result = print_config_structure({key: value}, prefix + '  ')
                result.extend(sub_result)
            else:
                # São arquivos
                for file in value:
                    result.append(f"{prefix}  {key}/{file}")
    
    return result

print("\nProposta de organização para arquivos de configuração:")
for line in print_config_structure(config_structure):
    print(line)

## 6. Atualização de Referências e Documentação

Ao mover arquivos, precisamos garantir que as referências a esses arquivos em outros lugares do código sejam atualizadas. Vamos criar uma estratégia para isso.

In [ ]:
# Função para encontrar referências aos arquivos que serão movidos
def find_references(file_patterns, search_dirs=['src', 'scripts', 'tests', 'notebooks']):
    """
    Procura referências aos arquivos que serão movidos no código-fonte.
    
    Args:
        file_patterns (list): Lista de padrões de arquivos para procurar
        search_dirs (list): Diretórios onde procurar
        
    Returns:
        dict: Dicionário com os arquivos e suas referências
    """
    references = {}
    
    for pattern in file_patterns:
        references[pattern] = []
        
        # Remove caracteres especiais como * que podem estar em padrões
        clean_pattern = pattern
        if '*' in clean_pattern:
            clean_pattern = clean_pattern.replace('*', '')
        
        # Comando grep (simulado)
        print(f"Procurando por referências a '{clean_pattern}'...")
        
        # Aqui fariamos uma busca no código-fonte
        # Exemplo simulado de resultado
        references[pattern].append({
            'file': f"exemplo/caminho/script.py",
            'line': 42,
            'content': f"with open('{clean_pattern}', 'r') as f:"
        })
    
    return references

# Simular busca de referências para alguns arquivos importantes
files_to_check = ['Dockerfile', 'nginx.conf.example', 'render.yaml', 'Procfile']
references = find_references(files_to_check)

# Exemplo de como lidar com essas referências
print("\nPossíveis atualizações de referências necessárias:")
for file, refs in references.items():
    if refs:
        print(f"\nArquivo: {file}")
        for ref in refs:
            print(f"  - {ref['file']}:{ref['line']} - {ref['content']}")
            new_path = file_movement_plan.get(file, 'unknown')
            if new_path:
                print(f"    Novo caminho sugerido: {new_path}")

# Estratégia para atualizar README.md
print("\nPlano para atualizar o README.md:")
print("1. Atualizar as referências a caminhos de arquivos")
print("2. Adicionar uma seção sobre a estrutura do projeto")
print("3. Atualizar instruções de instalação/implantação conforme necessário")

# Criação do novo conteúdo para o README.md sobre estrutura do projeto
readme_structure_section = """
## Estrutura do Projeto

```
fiap_tech_challenge_05/
├── .github/              # GitHub Actions e templates para issues/PRs
├── config/               # Arquivos de configuração
│   ├── docker/           # Arquivos Docker
│   ├── nginx/            # Configuração Nginx
│   └── render/           # Configuração da plataforma Render
├── data/                 # Dados do projeto
├── docs/                 # Documentação
├── examples/             # Exemplos de uso
├── logs/                 # Logs da aplicação
├── models/               # Modelos treinados
├── notebooks/            # Jupyter notebooks
├── scripts/              # Scripts utilitários
│   ├── deployment/       # Scripts de implantação
│   ├── monitoring/       # Scripts de monitoramento
│   └── utils/            # Scripts utilitários diversos
├── src/                  # Código-fonte principal
└── tests/                # Testes
```
"""

print("\nSeção sugerida para o README.md:")
print(readme_structure_section)

## 7. Script de Implementação

Vamos criar um script que pode automatizar a reorganização do projeto.

In [ ]:
# Script para reorganizar o projeto
reorganize_script = '''#!/usr/bin/env python3
"""
Script para reorganizar a estrutura do projeto Decision Scoring.
Este script implementa o plano de reorganização definido para melhorar
a organização do projeto, movendo arquivos para locais mais apropriados
e atualizando referências.

Uso:
    python reorganize_project.py [--dry-run] [--backup]

Opções:
    --dry-run   Apenas mostra o que seria feito, sem fazer alterações
    --backup    Cria um backup do projeto antes de fazer alterações
"""

import os
import sys
import shutil
import argparse
import glob
import re
from datetime import datetime
from pathlib import Path

# Configuração
PROJECT_ROOT = Path(__file__).resolve().parent
BACKUP_DIR = PROJECT_ROOT / "backup"

# Define o plano de movimentação de arquivos
FILE_MOVEMENT_PLAN = {
    # Arquivos Docker
    'Dockerfile': 'config/docker/api/Dockerfile',
    'Dockerfile.dashboard': 'config/docker/dashboard/Dockerfile',
    'docker-compose.yml': 'config/docker/docker-compose.yml',
    
    # Configuração Nginx
    'nginx.conf.example': 'config/nginx/nginx.conf',
    
    # Configuração Render
    'render.yaml': 'config/render/render.yaml',
    'Procfile': 'config/render/Procfile',
    '.env.render': 'config/render/.env.render',
    
    # Scripts
    'debug_api.py': 'scripts/utils/debug_api.py',
    'quick_deploy.sh': 'scripts/deployment/quick_deploy.sh',
    'commit-changes.sh': 'scripts/utils/commit-changes.sh',
    'generate_prospects_curl.py': 'scripts/utils/generate_prospects_curl.py',
    
    # Logs
    'api_logs.log': 'logs/api_logs.log',
}

# Arquivos a serem removidos (temporários, etc.)
FILES_TO_REMOVE = [
    'test_report.html',
]

# Estrutura de diretórios a ser criada
DIRECTORY_STRUCTURE = [
    '.github/workflows',
    'config/docker/api',
    'config/docker/dashboard',
    'config/nginx',
    'config/render',
    'logs',
    'scripts/deployment',
    'scripts/monitoring',
    'scripts/utils',
]

def parse_args():
    parser = argparse.ArgumentParser(description='Reorganiza a estrutura do projeto')
    parser.add_argument('--dry-run', action='store_true', help='Não faz alterações reais')
    parser.add_argument('--backup', action='store_true', help='Cria backup antes das alterações')
    return parser.parse_args()

def create_backup(dry_run=False):
    """Cria um backup do projeto."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_path = BACKUP_DIR / f"backup_{timestamp}"
    
    print(f"Criando backup em {backup_path}")
    if not dry_run:
        backup_path.mkdir(parents=True, exist_ok=True)
        
        # Copiar todos os arquivos exceto pastas grandes/desnecessárias
        for item in PROJECT_ROOT.glob('*'):
            if item.name in ['.git', '.venv', 'backup', '__pycache__']:
                continue
                
            if item.is_dir():
                shutil.copytree(item, backup_path / item.name)
            else:
                shutil.copy2(item, backup_path)
    
    return backup_path

def create_directory_structure(dry_run=False):
    """Cria a estrutura de diretórios necessária."""
    print("Criando estrutura de diretórios...")
    
    for dir_path in DIRECTORY_STRUCTURE:
        full_path = PROJECT_ROOT / dir_path
        print(f"  Criando diretório: {dir_path}")
        
        if not dry_run:
            full_path.mkdir(parents=True, exist_ok=True)

def move_files(dry_run=False):
    """Move os arquivos conforme o plano de movimentação."""
    print("Movendo arquivos...")
    
    for src_file, dest_path in FILE_MOVEMENT_PLAN.items():
        src_path = PROJECT_ROOT / src_file
        dest_full_path = PROJECT_ROOT / dest_path
        
        if not src_path.exists():
            print(f"  AVISO: Arquivo de origem não encontrado: {src_file}")
            continue
        
        print(f"  Movendo: {src_file} -> {dest_path}")
        
        if not dry_run:
            # Garantir que o diretório de destino exista
            dest_full_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Mover o arquivo
            shutil.move(src_path, dest_full_path)

def remove_files(dry_run=False):
    """Remove arquivos temporários ou desnecessários."""
    print("Removendo arquivos temporários...")
    
    for pattern in FILES_TO_REMOVE:
        for file_path in PROJECT_ROOT.glob(pattern):
            print(f"  Removendo: {file_path.relative_to(PROJECT_ROOT)}")
            
            if not dry_run:
                file_path.unlink()

def update_references(dry_run=False):
    """Atualiza referências aos arquivos movidos em outros arquivos."""
    print("Atualizando referências nos arquivos...")
    
    # Lista de diretórios onde procurar referências
    search_dirs = ['src', 'scripts', 'tests', 'notebooks']
    
    # Para cada arquivo movido, procura referências
    for src_file, dest_path in FILE_MOVEMENT_PLAN.items():
        # Pula alguns arquivos que não precisam ter referências atualizadas
        if src_file.endswith(('.log')):
            continue
            
        print(f"  Procurando referências a: {src_file}")
        
        # Constrói os padrões de busca (caminhos relativos e absolutos)
        search_patterns = [
            src_file,
            f"./{src_file}",
            str(PROJECT_ROOT / src_file)
        ]
        
        for search_dir in search_dirs:
            dir_path = PROJECT_ROOT / search_dir
            
            if not dir_path.exists():
                continue
                
            # Procura em todos os arquivos de código
            for ext in ['.py', '.sh', '.md', '.yml', '.yaml', '.json']:
                for code_file in dir_path.glob(f"**/*{ext}"):
                    try:
                        # Lê o conteúdo do arquivo
                        content = code_file.read_text()
                        updated = False
                        
                        # Verifica se há referências ao arquivo
                        for pattern in search_patterns:
                            if pattern in content:
                                print(f"    Encontrada referência em: {code_file.relative_to(PROJECT_ROOT)}")
                                
                                # Substitui o caminho antigo pelo novo
                                updated_content = content.replace(pattern, dest_path)
                                
                                if not dry_run and updated_content != content:
                                    code_file.write_text(updated_content)
                                    updated = True
                        
                        if updated:
                            print(f"    Atualizado: {code_file.relative_to(PROJECT_ROOT)}")
                    except Exception as e:
                        print(f"    ERRO ao processar {code_file}: {e}")

def update_readme(dry_run=False):
    """Atualiza o README.md com a nova estrutura do projeto."""
    readme_path = PROJECT_ROOT / "README.md"
    
    if not readme_path.exists():
        print("AVISO: README.md não encontrado")
        return
    
    print("Atualizando README.md...")
    
    try:
        readme_content = readme_path.read_text()
        
        # Seção para adicionar/atualizar
        structure_section = """
## Estrutura do Projeto

```
fiap_tech_challenge_05/
├── .github/              # GitHub Actions e templates para issues/PRs
├── config/               # Arquivos de configuração
│   ├── docker/           # Arquivos Docker
│   ├── nginx/            # Configuração Nginx
│   └── render/           # Configuração da plataforma Render
├── data/                 # Dados do projeto
├── docs/                 # Documentação
├── examples/             # Exemplos de uso
├── logs/                 # Logs da aplicação
├── models/               # Modelos treinados
├── notebooks/            # Jupyter notebooks
├── scripts/              # Scripts utilitários
│   ├── deployment/       # Scripts de implantação
│   ├── monitoring/       # Scripts de monitoramento
│   └── utils/            # Scripts utilitários diversos
├── src/                  # Código-fonte principal
└── tests/                # Testes
```
"""
        
        # Procura por uma seção existente sobre a estrutura do projeto
        structure_pattern = re.compile(r'#+\s*Estrutura\s+do\s+Projeto.*?(?=#+|$)', re.DOTALL)
        match = structure_pattern.search(readme_content)
        
        if match:
            # Atualiza a seção existente
            updated_readme = readme_content[:match.start()] + "## Estrutura do Projeto" + structure_section + readme_content[match.end():]
        else:
            # Adiciona uma nova seção no final
            updated_readme = readme_content + "\n\n" + "## Estrutura do Projeto" + structure_section
        
        # Atualiza também referências a caminhos
        for src_file, dest_path in FILE_MOVEMENT_PLAN.items():
            updated_readme = updated_readme.replace(src_file, dest_path)
        
        if not dry_run:
            readme_path.write_text(updated_readme)
            print("  README.md atualizado com sucesso")
    
    except Exception as e:
        print(f"ERRO ao atualizar README.md: {e}")

def create_contributing_file(dry_run=False):
    """Cria um arquivo CONTRIBUTING.md com diretrizes para contribuição."""
    contributing_path = PROJECT_ROOT / "CONTRIBUTING.md"
    
    # Verifica se o arquivo já existe
    if contributing_path.exists():
        print("AVISO: CONTRIBUTING.md já existe, pulando criação")
        return
    
    print("Criando CONTRIBUTING.md...")
    
    contributing_content = """# Contribuindo para o Projeto Decision Scoring

Agradecemos seu interesse em contribuir para o projeto! Aqui estão algumas diretrizes para ajudar você a começar.

## Fluxo de Trabalho

1. Faça fork do repositório
2. Clone o fork para sua máquina local
3. Configure o repositório upstream
4. Crie um branch para suas alterações
5. Faça suas alterações
6. Envie suas alterações para seu fork
7. Crie um Pull Request

## Convenções de Código

- Use snake_case para nomes de variáveis e funções
- Use PascalCase para nomes de classes
- Use UPPER_CASE para constantes
- Siga o padrão PEP 8 para código Python

## Estrutura do Projeto

Respeite a estrutura de diretórios do projeto:

- Código-fonte principal vai em `src/`
- Scripts utilitários vão em `scripts/`
- Configurações vão em `config/`
- Arquivos de log vão em `logs/`

## Commits

Use mensagens de commit claras e descritivas, seguindo o padrão:

```
tipo: descrição concisa

Descrição mais detalhada se necessário.
```

Onde `tipo` pode ser:
- `feat`: Nova funcionalidade
- `fix`: Correção de bug
- `docs`: Alterações na documentação
- `style`: Formatação, ponto e vírgula, etc; sem alteração de código
- `refactor`: Refatoração de código
- `test`: Adicionando testes, refatorando testes
- `chore`: Atualizações de tarefas de build, configurações, etc

## Testes

Certifique-se de adicionar testes para qualquer funcionalidade nova ou corrigida.
Execute a suite de testes antes de enviar seu Pull Request.

```bash
./scripts/run_tests.sh
```

## Documentação

Atualize a documentação relevante para suas alterações:

- README.md para alterações de uso
- Docstrings para funções e classes
- Comentários para código complexo
"""

    if not dry_run:
        contributing_path.write_text(contributing_content)
        print("  CONTRIBUTING.md criado com sucesso")

def main():
    args = parse_args()
    dry_run = args.dry_run
    
    if dry_run:
        print("MODO DE SIMULAÇÃO: Nenhuma alteração será feita")
    
    # Criar backup se solicitado
    if args.backup:
        backup_path = create_backup(dry_run)
        print(f"Backup criado em: {backup_path}")
    
    # Executar reorganização
    create_directory_structure(dry_run)
    move_files(dry_run)
    remove_files(dry_run)
    update_references(dry_run)
    update_readme(dry_run)
    create_contributing_file(dry_run)
    
    print("\nReorganização concluída!")
    if dry_run:
        print("Este foi apenas um teste, nenhuma alteração real foi feita.")
        print("Execute novamente sem --dry-run para aplicar as alterações.")

if __name__ == "__main__":
    main()
'''

print("Script de reorganização do projeto:")
print(reorganize_script)

## 8. Teste e Verificação

Finalmente, vamos definir alguns testes para verificar se a reorganização foi bem-sucedida.

In [ ]:
# Script para verificar a reorganização
verification_script = '''#!/usr/bin/env python3
"""
Script para verificar se a reorganização do projeto foi bem-sucedida.
Executa uma série de verificações para garantir que todos os arquivos
foram movidos corretamente e que o projeto continua funcionando.
"""

import os
import sys
from pathlib import Path
import importlib
import subprocess
import json

# Configuração
PROJECT_ROOT = Path(__file__).resolve().parent

def check_directory_structure():
    """Verifica se a estrutura de diretórios esperada existe."""
    print("Verificando estrutura de diretórios...")
    
    expected_dirs = [
        '.github/workflows',
        'config/docker/api',
        'config/docker/dashboard',
        'config/nginx',
        'config/render',
        'logs',
        'scripts/deployment',
        'scripts/monitoring',
        'scripts/utils',
        'src',
        'tests',
    ]
    
    all_exist = True
    for dir_path in expected_dirs:
        full_path = PROJECT_ROOT / dir_path
        if not full_path.exists() or not full_path.is_dir():
            print(f"  ERRO: Diretório não encontrado: {dir_path}")
            all_exist = False
        else:
            print(f"  OK: {dir_path}")
    
    return all_exist

def check_moved_files():
    """Verifica se os arquivos foram movidos para os locais corretos."""
    print("Verificando arquivos movidos...")
    
    expected_files = {
        'config/docker/api/Dockerfile': True,
        'config/docker/dashboard/Dockerfile': True,
        'config/docker/docker-compose.yml': True,
        'config/nginx/nginx.conf': True,
        'config/render/render.yaml': True,
        'config/render/Procfile': True,
        'scripts/utils/debug_api.py': True,
        'scripts/deployment/quick_deploy.sh': True,
        'logs/api_logs.log': False,  # Opcional, pode não existir
    }
    
    # Arquivos que não devem mais estar na raiz
    root_files_should_not_exist = [
        'Dockerfile',
        'Dockerfile.dashboard',
        'docker-compose.yml',
        'nginx.conf.example',
        'render.yaml',
        'Procfile',
        'debug_api.py',
        'quick_deploy.sh',
        'test_report.html',
    ]
    
    all_correct = True
    
    # Verifica arquivos movidos
    for file_path, required in expected_files.items():
        full_path = PROJECT_ROOT / file_path
        if not full_path.exists():
            if required:
                print(f"  ERRO: Arquivo esperado não encontrado: {file_path}")
                all_correct = False
            else:
                print(f"  AVISO: Arquivo opcional não encontrado: {file_path}")
        else:
            print(f"  OK: {file_path}")
    
    # Verifica arquivos que não devem estar na raiz
    for file_name in root_files_should_not_exist:
        root_file = PROJECT_ROOT / file_name
        if root_file.exists():
            print(f"  ERRO: Arquivo ainda existe na raiz: {file_name}")
            all_correct = False
    
    return all_correct

def check_imports():
    """Verifica se os imports do projeto ainda funcionam."""
    print("Verificando imports do projeto...")
    
    # Lista de módulos para testar
    modules_to_check = [
        'src.api',
        'src.data',
        'src.features',
        'src.models',
    ]
    
    all_imports_ok = True
    for module_name in modules_to_check:
        try:
            # Adiciona o diretório do projeto ao sys.path se necessário
            if str(PROJECT_ROOT) not in sys.path:
                sys.path.insert(0, str(PROJECT_ROOT))
                
            # Tenta importar o módulo
            importlib.import_module(module_name)
            print(f"  OK: {module_name}")
        except ImportError as e:
            print(f"  ERRO: Falha ao importar {module_name}: {e}")
            all_imports_ok = False
    
    return all_imports_ok

def run_tests():
    """Executa os testes do projeto para garantir que tudo ainda funciona."""
    print("Executando testes do projeto...")
    
    test_script = PROJECT_ROOT / 'scripts' / 'run_tests.sh'
    if not test_script.exists():
        print("  AVISO: Script de testes não encontrado")
        return None
    
    try:
        # Executa os testes
        result = subprocess.run(
            ['bash', str(test_script)],
            capture_output=True,
            text=True,
            check=False
        )
        
        # Verifica o resultado
        if result.returncode == 0:
            print("  OK: Todos os testes passaram")
            return True
        else:
            print(f"  ERRO: Alguns testes falharam (código {result.returncode})")
            print(f"  Saída: {result.stdout[:500]}...")  # Mostra parte da saída
            return False
    except Exception as e:
        print(f"  ERRO ao executar testes: {e}")
        return False

def print_summary(results):
    """Imprime um resumo dos resultados da verificação."""
    print("\n=== RESUMO DA VERIFICAÇÃO ===")
    
    all_passed = all(result for result in results.values() if result is not None)
    
    for check, result in results.items():
        status = "OK" if result else "FALHOU" if result is False else "PULADO"
        print(f"  {check}: {status}")
    
    if all_passed:
        print("\nSUCESSO! A reorganização foi concluída com êxito.")
    else:
        print("\nALERTA! A reorganização pode ter problemas que precisam ser corrigidos.")
    
def main():
    results = {}
    
    # Executar verificações
    results['estrutura_diretorios'] = check_directory_structure()
    results['arquivos_movidos'] = check_moved_files()
    results['imports'] = check_imports()
    results['testes'] = run_tests()
    
    # Imprimir resumo
    print_summary(results)
    
    # Retornar código de saída
    return 0 if all(result for result in results.values() if result is not None) else 1

if __name__ == "__main__":
    sys.exit(main())
'''

print("Script de verificação da reorganização:")
print(verification_script)

## Conclusão

Neste notebook, analisamos a estrutura atual do projeto Decision Scoring, identificamos problemas de organização e desenvolvemos um plano detalhado para melhorá-la. 

Os principais pontos abordados foram:

1. **Análise da estrutura atual**: Escaneamos o projeto e visualizamos a distribuição de arquivos.
2. **Identificação de problemas**: Encontramos arquivos de configuração, scripts e código dispersos na raiz do projeto.
3. **Proposta de estrutura padrão**: Definimos uma estrutura de diretórios seguindo boas práticas para projetos Python.
4. **Plano de movimentação**: Criamos um mapeamento detalhado de onde cada arquivo deve ser movido.
5. **Gestão de configurações**: Propusemos uma estrutura para organizar arquivos de configuração.
6. **Atualização de referências**: Criamos estratégias para atualizar referências aos arquivos movidos.
7. **Scripts de implementação**: Desenvolvemos scripts para automatizar a reorganização e validar o resultado.

A implementação dessas mudanças tornará o projeto mais organizado, mais fácil de manter e seguindo padrões reconhecidos pela comunidade Python.

In [ ]:
# Salvando os scripts para implementação
import os

# Criando diretório para scripts de reorganização
scripts_dir = os.path.join(project_root, 'scripts', 'utils', 'reorganization')
os.makedirs(scripts_dir, exist_ok=True)

# Salvando o script de reorganização
with open(os.path.join(scripts_dir, 'reorganize_project.py'), 'w') as f:
    f.write(reorganize_script)

# Salvando o script de verificação
with open(os.path.join(scripts_dir, 'verify_reorganization.py'), 'w') as f:
    f.write(verification_script)

print(f"Scripts salvos no diretório: {scripts_dir}")
print("Para executar a reorganização (em modo de simulação):")
print(f"python {os.path.join(scripts_dir, 'reorganize_project.py')} --dry-run")
print("\nPara executar a reorganização real:")
print(f"python {os.path.join(scripts_dir, 'reorganize_project.py')}")
print("\nPara verificar após a reorganização:")
print(f"python {os.path.join(scripts_dir, 'verify_reorganization.py')}")